# Aerogel Bonding AI & ML Project 2024

## Importing Libraries & Reading Data

In [39]:
#Importing libraries 

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Reading dataset
path = './aerogel_bonding.csv'
starting_data = pd.read_csv(path)

## EDA & Data Pre-Processing

This section explores the aerogel bonding dataset through Exploratory Data Analysis (EDA).

### Understanding dataset

First, configure pandas to display all columns in the DataFrame.

In [40]:
pd.set_option('display.max_columns', None)

#### Vizualising and understaning data structure & values

Now, let's visualize the first few rows of the dataset to get a better understanding of its structure.

In [41]:
starting_data.head()

,HolidaysTaken,PercentageOfCompletedTasks,CurrentJobDuration,RecentHolidaysTaken,RequestedProcessAmount,JobStatus,BondingRiskRating,TotalMaterialProcessed,ByproductRation,working_skills,CivilStatus,dependability,MistakesLastYear,HighestEducationAttained,BondingSuccessful,ChurnRisk,ProcessedKilograms,SkillRating,ProcessingTimestamp,WorkExperience,HistoricalBehavior,TotalMaterialToProcess,WorkHistoryDuration,ApplicantAge,PriorExecutionDefaults,DifferentTasksCompleted,TotalChurnRisk,OtherCompaniesMaterialProcessed,BondingPeriod,trustability,MonthlyExecutions
0,4.0,NaN,6.0,1.0,51172.0,Employed,46.0,300388.0,0.273137,1.244120,Married,2.798099,0.0,Master,0.0,0.223172,44305.0,606.0,2055-07-29,48.0,23.0,14193.0,3.0,67.0,0.0,NaN,NaN,NaN,24.0,3.144274,440.0
1,1.0,0.323046,4.0,0.0,11246.0,Employed,54.0,299914.0,0.450387,2.228183,Married,2.586440,0.0,Bachelor,NaN,0.215746,NaN,561.0,2072-03-10,31.0,17.0,85355.0,28.0,52.0,NaN,0.0,0.232582,214559.0,36.0,3.704809,NaN
2,5.0,0.491574,3.0,1.0,14075.0,Employed,42.4,74687.0,0.325027,2.699264,Married,1.949641,0.0,Bachelor,1.0,0.256075,67954.0,NaN,2032-01-24,20.0,25.0,14006.0,NaN,45.0,0.0,0.0,0.240812,60681.0,60.0,2.427195,171.0
3,4.0,0.108916,3.0,1.0,18957.0,Employed,40.8,47866.0,NaN,0.445854,Married,1.569581,0.0,Bachelor,1.0,0.240457,98184.0,607.0,2029-11-12,19.0,32.0,13240.0,16.0,42.0,0.0,4.0,0.231520,34626.0,84.0,1.156431,212.0
4,NaN,0.174628,1.0,2.0,17902.0,Employed,51.0,18181.0,0.388317,1.940075,Single,2.149917,NaN,Associate,NaN,0.206902,48981.0,612.0,2031-08-22,28.0,14.0,44217.0,28.0,50.0,0.0,0.0,0.214425,4812.0,48.0,3.185402,323.0


This visual representation of the data makes it easier to spot patterns and potential anomalies.

To more efficiently identify all data threats, we will conduct a detailed analysis of each column.

#### Deeper evaluation on dataset features

- **HolidaysTaken:** Integer of days taken for holiday by the worker, it can be NaN if the worker didn't take any days off, same as it would be for 0, it wouldn't affect any calculations, only rows with negative numbers if found should be excluded.
- **PercentageOfCompletedTasks:** Float between 0 and 1, representing percentage(divided by 100) of completed tasks by worker, it can be NaN in case someone hasn't completed any given task, or in case it hasn't been assigned any work
- **CurrentJobDuration:** Integer representing number of months worker has been working there, in case of NaN, negative or 0, row should be excluded.
- **RecentHolidaysTaken:** Integer of days taken for recent holiday, it should be smaller or equal to HolidaysTaken, otherwise row should be excluded.
- **RequestedProcessAmount:** Integer of amount of material requested for bonding process, NaN and anything smaller than 0 should be excluded.
- **JobStatus:** Enum (object) for status of worker, in order to represent it we will be using One-Hot Encoding as we don't want to impose any ordering between categories, additionally it wouldn't impose big issues regarding memory usage as our dataset is not so big and we just have three categories, which makes it very suitable for this problem. Additionally, all NaNs should be excluded.
- **BondingRiskRating:** Float between 0 and 100, indicating percentage associated to risk between bonding using certain materials or processes, NaNs can be used as zeros, anything out of range 0-100 should be consider as error and excluded.
- **TotalMaterialProcessed:** Integer representing cumulative amount of materials used by worker or team of workers through bonding processes, zeros and negatives should be excluded.
- **ByproductRation:** Float between 0 and 1, representing percentage(divided by 100) of byproducts generated bonding relative to total number of materials used, anything out of range or NaNs can be excluded.
- **working_skills:** Float between 0 and 5, representing rating of a worker's skills, 
- **CivilStatus:** 
- **dependability:**

#### Dataset description

In [43]:
print(f"Number of rows and columns: {starting_data.shape}")
print("Data Types and Missing Values:")
print(starting_data.info())

Number of rows and columns: (20000, 31)
Data Types and Missing Values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HolidaysTaken                    18095 non-null  float64
 1   PercentageOfCompletedTasks       17955 non-null  float64
 2   CurrentJobDuration               17952 non-null  float64
 3   RecentHolidaysTaken              18031 non-null  float64
 4   RequestedProcessAmount           17925 non-null  float64
 5   JobStatus                        17982 non-null  object 
 6   BondingRiskRating                17950 non-null  float64
 7   TotalMaterialProcessed           17957 non-null  float64
 8   ByproductRation                  18028 non-null  float64
 9   working_skills                   18034 non-null  float64
 10  CivilStatus                      18046 non-null  object 
 11  dependabi

### Checking data integrity

#### Identifying missing values

In [19]:
# Configure pandas to display the entire table without line breaks
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Set the width to prevent line breaks

# Calculate missing values
missing_values = df.isnull().sum()

# Calculate total values (missing + non-missing)
total_values = df.shape[0]  # Total rows in the DataFrame

# Calculate percentage of missing values and format with percentage sign
percentage_missing = (missing_values / total_values * 100).apply(lambda x: f'{x:.2f}%')

# Add data types
data_types = df.dtypes

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Data Type': data_types,
    'Missing Values': missing_values,
    'Total Values': total_values,
    'Percentage Missing': percentage_missing,
})

# Sort by Missing Values
summary_df.sort_values(by='Missing Values', ascending=False, inplace=True)

# Print the summary
print("Summary:")
print(summary_df)

# Reset pandas options after printing if needed
pd.reset_option('display.max_columns')
pd.reset_option('display.width')


Summary:
                           Data Type  Missing Values  Total Values Percentage Missing
PercentageOfCompletedTasks   float64               0          2466              0.00%
CurrentJobDuration           float64               0          2466              0.00%
RequestedProcessAmount       float64               0          2466              0.00%
BondingRiskRating            float64               0          2466              0.00%
TotalMaterialProcessed       float64               0          2466              0.00%
ByproductRation              float64               0          2466              0.00%
working_skills               float64               0          2466              0.00%
dependability                float64               0          2466              0.00%
MistakesLastYear             float64               0          2466              0.00%
BondingSuccessful            float64               0          2466              0.00%
ProcessedKilograms           float64         

#### Missing rates and distribution of missing rates

#### Handling of missing data

In [20]:
#MAYBE DO SHORTER


# Calculate the number of duplicate rows
duplicates = df.duplicated().sum()

# Calculate percentage of duplicates
total_rows = df.shape[0]
percentage_duplicates = (duplicates / total_rows) * 100

# Print summary of duplicates
print(f"Total duplicate rows: {duplicates}")
print(f"Percentage of duplicates: {percentage_duplicates:.2f}%")

# Optionally display the duplicate rows
if duplicates > 0:
    print("Duplicate rows:")
    print(df[df.duplicated()])
else:
    print("No duplicate rows found.")


Total duplicate rows: 0
Percentage of duplicates: 0.00%
No duplicate rows found.


#### Analysis of potential irrelevant data

In [21]:
# Columns identified as irrelevant
columns_to_drop = [
    'HolidaysTaken', 
    'RecentHolidaysTaken', 
    'JobStatus', 
    'CivilStatus', 
    'HighestEducationAttained', #very very small
    'ChurnRisk', #very small
    'ProcessingTimestamp', #medium 
    'ApplicantAge', #reduntant we have work experience
    'WorkHistoryDuration', #reduntant we have work experience
    'TotalChurnRisk', #we have churn risk ???
    'OtherCompaniesMaterialProcessed' #???
]

# Drop irrelevant columns
df = df.drop(columns=columns_to_drop, errors='ignore')

# Drop rows with missing values
df = df.dropna()

# Display summary after dropping
print("Updated DataFrame Info:")
print(df.info())


Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 2466 entries, 8 to 19992
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PercentageOfCompletedTasks  2466 non-null   float64
 1   CurrentJobDuration          2466 non-null   float64
 2   RequestedProcessAmount      2466 non-null   float64
 3   BondingRiskRating           2466 non-null   float64
 4   TotalMaterialProcessed      2466 non-null   float64
 5   ByproductRation             2466 non-null   float64
 6   working_skills              2466 non-null   float64
 7   dependability               2466 non-null   float64
 8   MistakesLastYear            2466 non-null   float64
 9   BondingSuccessful           2466 non-null   float64
 10  ProcessedKilograms          2466 non-null   float64
 11  SkillRating                 2466 non-null   float64
 12  WorkExperience              2466 non-null   float64
 13  HistoricalBeh

### Data profiling and statistical overview

#### Data distribution across categorical, discrete, and continuous variables

**Thing to point out :** distribution of succeded bonding

##### Distribution of discrete numerical values

##### Distribution of continous numerical values

##### Distribution of categorical values

#### Label encoding for target value 

### Correlation heatmap

#### Preprocessing dataset for expected heatmap 

#### Final correlation heatmap of cleaned and scaled dataframe

## CHECK TO ADD SOMETHING (data splitting, preparing data for modelling, feature selection)

## Machine Learning Models

### Model selection

### Model training and evaluation

### Possible resampling ??

###  Hyperparameter tuning

### Model Comparison and Feature Importance and Interpretability ???

### Evaluation and analysis of models

## Model Results and Evaluation

## Conclusion